In [43]:
import os
import xml.etree.ElementTree as ET 
import struct
import numpy as np
import cv2
import random
from IPython.core.debugger import Tracer
from cfgs.config import cfg

In [44]:
def parse_xml(data_root,xml_file_path):
    if os.path.isfile(xml_file_path) == False:
        return [],[]
    tree = ET.parse(xml_file_path)
    root = tree.getroot()
    
    image_paths = []
    labels = []

    images = root.find("images")
    images = images.findall("image")
    
    for image in images:
        image_name = image.get("file").split('\\')[-1]
        image_path = os.path.join(data_root,image_name)
#         print(image_path)
        cur_img = cv2.imread(image_path)
        if cur_img is None:
            print("image no size:" + image_path)
            continue
        img_height,img_width,img_channel= cur_img.shape
        
        image_labels = []
        boxes = image.findall("box")
        if len(boxes) == 0:
            print("image no boxes:" + image_path)
            continue
        for box in boxes:
            top = int(box.get("top"))
            left = int(box.get("left"))
            width = int(box.get("width"))
            height = int(box.get("height"))
            
            xmin = np.max([left,0])
            xmax = np.min([left + width,img_width])
            ymin = np.max([0,top])
            ymax = np.min([top + height,img_height])
            
            image_labels.append([xmin,ymin,xmax,ymax,0])

        if len(image_labels) == 0:
            continue
        image_paths.append(os.path.join(data_root,image_name))
        labels.append(image_labels)
#     print(len(image_paths))
#     print(len(labels))
    return image_paths,labels

In [45]:
data_dirs = os.listdir(cfg.ROOT_DATA_DIR)

In [46]:
image_paths = []
labels = []
for data_dir in data_dirs:
    if data_dir.endswith("_crop") == True:
        continue
    data_dir_path_ = os.path.join(cfg.ROOT_DATA_DIR,data_dir)
    if os.path.isdir(data_dir_path_) == False:
        continue
    dir_item = os.listdir(data_dir_path_)   
    for dir_ in dir_item:
        data_dir_paths = os.path.join(data_dir_path_,dir_)
        if os.path.isdir(data_dir_paths) == False:
            continue
        class_dirs = os.listdir(data_dir_paths)
        for class_dir in (class_dirs):
            class_path = os.path.join(data_dir_paths,class_dir)
            if os.path.isdir(class_path) == False:
                continue
            img_dirs = os.listdir(class_path)
            for img_dir in img_dirs:
                if not img_dir.endswith('xml'):
                    img_xmlpath = os.path.join(class_path,img_dir)
                    label_path = img_xmlpath + '.xml'
#                     print(img_xmlpath)
#                     print(label_path)
                    cur_path,cur_label = parse_xml(img_xmlpath, label_path)
                    image_paths.extend(cur_path)         
                    labels.extend(cur_label)
    #                 print(image_paths)
    #                 print(labels)
#                     break
#             break
#         break
#     break

In [47]:
#convert_to_string(imagePath,imageLabel)
def convert_to_string(image_path,labels):
    out_string = ''
    out_string += image_path
    for label in labels:
        for i in label:
            out_string += ' ' + str(i)
    out_string += '\n'
#     print(out_string)
    return out_string

records = []
for id, path in enumerate(image_paths):
    label=labels[id]
    records.append(convert_to_string(path,label))
random.shuffle(records)
print("total size:",len(records))
# print(records[0:5])

total size: 2997


In [48]:
total_num = len(records)
test_num = int(cfg.test_ratio*total_num)
train_num = total_num - test_num
train_records = records[0:train_num]
tests_records = records[train_num:]

In [49]:
train_out_file = open(cfg.train_file_name,'w')
for record in train_records:
    train_out_file.write(record)
train_out_file.close()


test_out_file = open(cfg.test_file_name,'w')
for record in tests_records:
    test_out_file.write(record)
test_out_file.close()